In [160]:
import pathlib
import pandas as pd
import xml.etree.ElementTree as et
import numpy as np

# define the path
currentDirectory = pathlib.Path('data/Test_Set/XML')

# define the pattern
currentPattern = "*.xml"
all_xml_files = []
for currentFile in currentDirectory.glob(currentPattern):
    all_xml_files.append(currentFile.parts[-1])
    
print(all_xml_files)

['Sleeping_Beauty.xml', 'Foxcatcher.xml', 'The_Trip_to_Italy.xml', 'A_Little_Chaos.xml', 'Bad_News_Bears.xml', 'Run_Lola_Run.xml', 'Love_Actually.xml', '12_Years_a_Slave.xml', 'Kundo__Age_of_the_Rampant.xml', 'The_Best_of_Me.xml', 'A_Most_Wanted_Man.xml', 'The_Simpsons_Movie.xml', 'Homeland.xml', 'Bride_Flight.xml', 'Rise_of_the_Planet_of_the_Apes.xml', 'Banklady.xml', 'Silver_Linings_Playbook.xml', 'Godzilla.xml', 'Antz.xml', 'The_Hobbit__An_Unexpected_Journey.xml', 'The_Shawshank_Redemption.xml', 'Mr._Turner.xml', 'Hum_Aapke_Hain_Koun...!.xml', 'Legally_Blonde.xml', 'The_Inglorious_Bastards.xml', 'Soof.xml', 'Testament_of_Youth.xml', 'Dilwale_Dulhania_Le_Jayenge.xml', 'Prometheus.xml', 'Maps_to_the_Stars.xml', 'Bewakoofiyaan.xml', 'Big_Game.xml', 'Blow_Out.xml', 'Fast_Five.xml', 'Verliefd_op_Ibiza.xml', 'Leviathan.xml', 'The_English_Patient.xml', 'And_So_It_Goes.xml', 'Alice_in_Wonderland.xml', 'The_Great_Escape.xml', 'Smokey_and_the_Bandit.xml', 'Magic_in_the_Moonlight.xml', 'Edge_o

223

In [161]:
def parse_XML(xml_file, name, df_cols):
            global out_df

            xtree = et.parse(xml_file)
            xroot = xtree.getroot()
            fin = []
            res = []
            res.append(name[:-4])
            
            for node in xroot:
                for i in node_cols:
                    res.append(node.attrib.get(i))

            fin.append(res)
            print(fin)
            out_df = pd.DataFrame(data=fin, columns =meta_cols)
            return out_df
        
def parse_XML1(xml_file, name, df_cols, df):
            global out_df
            xtree = et.parse(xml_file)
            xroot = xtree.getroot()
            fin = []
            res = []
            res.append(name[:-4])
            for node in xroot:
                for i in node_cols:
                    res.append(node.attrib.get(i))

            fin.append(res)
            df1 = pd.DataFrame(data=fin, columns =meta_cols)
            
            frames = [df, df1]
            
            out_df = pd.concat(frames)
            
            return out_df

In [162]:
meta_cols = ["filename","language", "year", "genre", "country",
            "runtime", "rated"]

node_cols = ["language", "year", "genre", "country",
            "runtime", "rated"]

In [163]:
c = 1
for i in all_xml_files:
    if c==1:
        parse_XML("data/Test_Set/XML/" + i, i,  meta_cols)
        c=43
    else:
        parse_XML1("data/Test_Set/XML/"+i, i, meta_cols, out_df)

[['Sleeping_Beauty', 'English', '1959', 'Animation, Family, Fantasy', 'USA', '75 min', 'APPROVED']]


In [164]:
df_ground = pd.read_csv("data/Test_Set/test_set_labels.csv", delimiter=";", quoting = 3)
# Rename columns for joining afterwards
df_ground.columns = ["movie", "filename", "goodforairplane"]
# Only for TestLabels because 2 files contain .mp4 ending
df_ground["filename"] = df_ground["filename"].str.replace(".mp4","")
# Because 4 lines have "" in their names
df_ground["movie"] = df_ground["movie"].str.replace('"',"")
df_ground["goodforairplane"] = df_ground["goodforairplane"].str.replace('"',"")
df_ground["goodforairplane"] = df_ground["goodforairplane"].astype(int)
# Check for duplicates
df_ground[df_ground.duplicated(["filename"])]
# Remove duplicates
i = df_ground[df_ground["movie"] == "10.000_km"].index
df_ground = df_ground.drop(i)

In [165]:
a = df_ground["filename"].sort_values()
b = out_df["filename"].sort_values()

for i in a.values:
    if i in b.values:
        continue
    else:
        print(i)

A_Fish_Called_Wanda


In [166]:
df_join = pd.merge(df_ground, out_df, on="filename", how="inner")

In [167]:
df_join = df_join.drop(["movie", "filename"], axis=1)

In [168]:
import numpy as np

count=0
for i in df_join["genre"].values:
    df_join["genre"].values[count] = i.replace(" ", "")
    count+=1
    
count=0
for i in df_join["language"].values:
    df_join["language"].values[count] = i.replace(" ", "")
    count+=1
    
    
count=0
for i in df_join["country"].values:
    df_join["country"].values[count] = i.replace(" ", "")
    count+=1

In [169]:
genre = df_join.genre.str.split(",", expand=True).stack()
language = df_join.language.str.split(",", expand=True).stack()
country = df_join.country.str.split(",", expand=True).stack()

In [170]:
df_join = pd.concat([df_join, 
                pd.get_dummies(genre, prefix="g").groupby(level=0).sum()],axis=1) \
                .drop(["genre"],axis=1)

df_join = pd.concat([df_join, 
                pd.get_dummies(language, prefix="l").groupby(level=0).sum()],axis=1) \
                .drop(["language"],axis=1)

df_join = pd.concat([df_join, 
                pd.get_dummies(country, prefix="c").groupby(level=0).sum()],axis=1) \
                .drop(["country"],axis=1)

In [171]:
df_join.columns
df_join["runtime"] = df_join["runtime"].apply(lambda x: x.split(" min")[0])
df_join["runtime"] = df_join["runtime"].apply(pd.to_numeric, errors="coerce")

In [172]:
df_join.to_csv("metadata_test.csv",index=False)

goodforairplanes        0
year                    0
runtime                 2
rated                   0
g_Action                0
                       ..
c_Turkey                0
c_UK                    0
c_USA                   0
c_UnitedArabEmirates    0
c_WestGermany           0
Length: 101, dtype: int64